# XAI Fachvertiefung

Lecturer: Susanne Suter (susanne.suter@fhnw.ch)

**Bemerkung**: __[Dies ist eine modifizierte Kopie eines iNNvestigate Github Repository Notebooks](https://github.com/albermax/innvestigate/blob/master/examples/imagenet_compare_methods.ipynb)__ (2023-01-06)


<font color='blue'>Aufgaben Fachvertiefungsmodul: </font>

* Führe dieses adaptierte Notebook von iNNvestigate durch. 
Bemerkung: Damit du schneller Daten visualisieren kannst, empfiehlt es sich, die Netze für weniger Epochen laufen zu lassen und nicht alle XAI Methoden von Innvestigate berechnen zu lassen (deshalb sind einige Methoden auskommentiert)

* Beantworte folgende Fragen:
    * Was bedeuten LRP-z bzw. LRP-Epsilon? Stellst du Unterschiede fest?
    * Wofür ist die LRP-Methoden besonders gut geeignet? Im Vergleich zu den anderen Methoden?
* Lade eigene Beispielbilder in den entsprechenden Ordner und versuche Beispiele zu finden, bei welchen die Relevanz zur Vorhersage nicht den Erwartungen entspricht
* Vergleiche weiter von iNNvestigate implementierte XAI Methoden (Analysis Teil)
* Spiele mit weiteren Notebooks von __[iNNvestigate](https://github.com/albermax/innvestigate/tree/master/examples)__

# Layerwise Relevance Propagation (LRP)


LRP ist eine Methode, die für die Entscheidung relevante Pixel identifiziert. Dafür wird in einem Rückwärtsdurchlauf durchs neuronale Netz konservatives Relevanzumverteilungsverfahren eingesetzt, bei welchem die Neuronen, die am meisten zur vorangehenden Schicht beitragen, die meiste Relevanz von dieser erhalten. 


Links: 
* __[Konferenz-Tutorial zu LRP](http://www.interpretable-ml.org/ecml2020tutorial/ecml2020-xai-tutorial-2.pdf)__ (Folien 29-34)
* __[Chapter 10: Layer-Wise Relevance Propagation: An Overview](https://iphome.hhi.de/samek/pdf/MonXAI19.pdf)__
* __[Heatmapping.org Webseite](http://www.heatmapping.org/)__ (gute Quelle für viele Tutorials und verlinkte Repositories)
* __[Github Repository LRP](https://github.com/albermax/innvestigate)__ (TensorFlow - für PyTorch gibt es andere Bibliotheken; enthält diverse Beispiel-Notebooks)
* __[Git Repository mit LRP Tutorial](https://git.tu-berlin.de/gmontavon/lrp-tutorial)__ (inkl. Publikationshinweise und Implementierungsdetails)

# Before You Start

1) Click on "Copy to Drive" to work on your personal copy of the notebook. A Google account is needed for that.

2) Select "Runtime" -> "Change runtime type" -> "Hardware accelerator" : "GPU"

## Data Upload and Imports

You need to upload the __[utils](https://github.com/albermax/innvestigate/tree/master/examples/utils)__ and __[images](https://github.com/albermax/innvestigate/tree/master/examples/images)__ from the iNNvestigate repository. This can either be done via Google drive (persistent) or every time you start a Colab session via manual upload of files (see left tab at bottom).

In [ ]:
from google.colab import drive
import os
drive_dir = os.path.abspath('drive')
drive.mount(drive_dir)

## Install Libraries

In [ ]:
#downgrade the following libraries to be able to install iNNvestigate
!pip install numpy==1.22.0

In [ ]:
# Führe folgende Zeile aus, sofern du iNNvestigate noch nicht installiert hast. 
!pip install innvestigate

# Compare analyzers on ImageNet

In this notebook we show how one can use **iNNvestigate** to analyze the prediction of ImageNet-models! To do so we will load a network from the keras.applications module and analyze prediction on some images!

Parts of the code that do not contribute to the main focus are outsourced into utility modules. To learn more about the basic usage of **iNNvestigate** have look into this notebook: [Introduction to iNNvestigate](introduction.ipynb) and [Comparing methods on MNIST](mnist_method_comparison.ipynb)

-----

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
# warnings.simplefilter('ignore')

In [ ]:
%matplotlib inline

import imp
import os

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

tf.compat.v1.disable_eager_execution()

import innvestigate
import innvestigate.utils as iutils

In [ ]:
# Use utility libraries to focus on relevant iNNvestigate routines
utils_dir = os.path.join(drive_dir, 'MyDrive', 'xai_data', 'utils')

eutils = imp.load_source("utils", os.path.join(utils_dir, "__init__.py"))
imagenetutils = imp.load_source("utils_imagenet", os.path.join(utils_dir, "imagenet.py"))

## Model

In this demo use the VGG16-model, which uses ReLU activation layers.

In [ ]:
# Load the model definition.
model = VGG16(weights="imagenet")

# Handle input depending on model and backend.
channels_first = keras.backend.image_data_format() == "channels_first"
color_conversion = "BGRtoRGB"  # keras.applications use BGR format

## Data

Now we load some example images and preprocess them to fit the input size model.

To analyze your own example images, just add them to `innvestigate/examples/images`.

In [ ]:
# Get some example test set images.
image_shape = [224, 224]
images, label_to_class_name = eutils.get_imagenet_data(size=image_shape[0])

if not len(images):
    raise Exception(
        "Please download the example images using: "
        "'innvestigate/examples/images/wget_imagenet_2011_samples.sh'"
    )

## Analysis

Next, we will set up a list of analysis methods by preparing tuples containing the methods' string identifiers used by `innvestigate.analyzer.create_analyzer(...)`, some optional parameters, a post processing choice for visualizing the computed analysis and a title for the figure to render. Analyzers can be deactivated by simply commenting the corresponding lines, or added by creating a new tuple as below.

For a full list of methods refer to the dictionary `investigate.analyzer.analyzers`.

Note: Should you run into resource trouble, e.g. you are running that notebook on a computer without GPU or with only limited GPU memory, consider deactivating one or more analyzers by commenting the corresponding lines in the next cell.

In [ ]:
input_range = (-128, 128)  # format used by keras.applications
noise_scale = (input_range[1] - input_range[0]) * 0.1

# Methods we use and some properties.


methods = [
    # NAME                  OPT.PARAMS                  POSTPROC FXN            TITLE
    # Show input.
    ("input",               {},                         imagenetutils.image,    "Input"),
    # Function
    ("gradient",            {"postprocess": "abs"},     imagenetutils.graymap,  "Gradient"),
    #("smoothgrad",          {"augment_by_n": 64, 
    #                         "noise_scale": noise_scale, 
    #                         "postprocess": "square"},  imagenetutils.graymap,  "SmoothGrad"),
    # Signal
    #("deconvnet",           {},                         imagenetutils.bk_proj,  "Deconvnet"),
    #("guided_backprop",     {},                         imagenetutils.bk_proj,  "Guided Backprop"),
    # Interaction
    #("deep_taylor.bounded", {"low": input_range[0], 
    #                         "high": input_range[1]},   imagenetutils.heatmap,  "DeepTaylor"),
    #("input_t_gradient",    {},                         imagenetutils.heatmap,  "Input * Gradient"),
    #("integrated_gradients",
    #                        {"reference_inputs": input_range[0], 
    #                         "steps": 64},              imagenetutils.heatmap,  "Integrated Gradients"),
    ("lrp.z",               {},                         imagenetutils.heatmap,  "LRP-Z"),
    ("lrp.epsilon",         {"epsilon": 1},             imagenetutils.heatmap,  "LRP-Epsilon"),
    ("lrp.sequential_preset_a_flat",
                            {"epsilon": 1},             imagenetutils.heatmap,  "LRP-PresetAFlat"),
    ("lrp.sequential_preset_b_flat",
                            {"epsilon": 1},             imagenetutils.heatmap,  "LRP-PresetBFlat"),
]

The main loop below will now instantiate the analyzer objects based on the loaded/trained model and the analyzers' parameterizations above and compute the analyses.

In [ ]:
# Create model without trailing softmax
model_wo_softmax = innvestigate.model_wo_softmax(model)

# Create analyzers.
analyzers = []
for method in methods:
    try:
        analyzer = innvestigate.create_analyzer(
            method[0],  # analysis method identifier
            model_wo_softmax,  # model without softmax output
            **method[1]
        )  # optional analysis parameters
    except innvestigate.NotAnalyzeableModelException:
        # Not all methods work with all models.
        analyzer = None
    analyzers.append(analyzer)

Now we analyze each image with the different analyzers:

In [ ]:
analysis = np.zeros([len(images), len(analyzers)] + image_shape + [3])
text = []


for i, (x, y) in enumerate(images):
    # Add batch axis.
    x = x[None, :, :, :]
    x_pp = preprocess_input(x)

    # Predict final activations, probabilites, and label.
    presm = model_wo_softmax.predict_on_batch(x_pp)[0]
    prob = model.predict_on_batch(x_pp)[0]
    y_hat = prob.argmax()

    # Save prediction info:
    text.append(
        (
            "%s" % label_to_class_name[y],  # ground truth label
            "%.2f" % presm.max(),  # pre-softmax logits
            "%.2f" % prob.max(),  # probabilistic softmax output
            "%s" % label_to_class_name[y_hat],  # predicted label
        )
    )

    for aidx, analyzer in enumerate(analyzers):
        if methods[aidx][0] == "input":
            # Do not analyze, but keep not preprocessed input.
            a = x / 255
        elif analyzer:
            # Analyze.
            a = analyzer.analyze(x_pp)

            # Apply common postprocessing, e.g., re-ordering the channels for plotting.
            a = imagenetutils.postprocess(a, color_conversion, channels_first)
            # Apply analysis postprocessing, e.g., creating a heatmap.
            a = methods[aidx][2](a)
        else:
            a = np.zeros_like(image)
        # Store the analysis.
        analysis[i, aidx] = a[0]

Next, we visualize the analysis results:

In [ ]:
# Prepare the grid as rectangular list
grid = [
    [analysis[i, j] for j in range(analysis.shape[1])] for i in range(analysis.shape[0])
]
# Prepare the labels
label, presm, prob, pred = zip(*text)
row_labels_left = [
    ("label: {}".format(label[i]), "pred: {}".format(pred[i]))
    for i in range(len(label))
]
row_labels_right = [
    ("logit: {}".format(presm[i]), "prob: {}".format(prob[i]))
    for i in range(len(label))
]
col_labels = ["".join(method[3]) for method in methods]

# Plot the analysis.
eutils.plot_image_grid(
    grid,
    row_labels_left,
    row_labels_right,
    col_labels,
    file_name=os.environ.get("plot_file_name", None),
)

This figure shows the analysis regarding the *actually predicted* class as computed by the selected analyzers. Each column shows the visualized results for different analyzers and each row shows the analyses wrt to one input sample. To the left of each row, the ground truth label `label` and the predicted label `pred` are show. To the right, the model's probabilistic (softmax) output is shown as `prob` and the logit output just before the terminating softmax layer as `logit`. Note that all analyses have been performed based on the logit output (layer).


If you are curious about how **iNNvestigate** performs on *different* ImageNet model, have a look here: [Comparing networks on ImageNet](imagenet_network_comparison.ipynb)